# Plugins

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rxavier/poniard/blob/master/examples/05._plugins.ipynb)

This notebook explains how Poniard's extensible plugin system works.

If you don't have it installed, please install from PyPI.

In [1]:
# %pip install poniard

Plugins can be thought of as callbacks in other machine learning frameworks (Keras, fastai, PyTorch Lightning, etc.). Generally, they are objects that do things at given points in time, for example, after each training epoch ends. They generally do not modify the existing behavior of the framework, but instead add new functionality.

These objects in Poniard are deliberetely not called callbacks because there is a priori no restriction over what they can do. However, the main concept remains the same.

In essence, plugin hooks are executed at different points during a Poniard estimator's lifetime, for example `self._run_plugin_method("on_setup_end")`. These hooks check which plugins, if any, were set during initialization, and run the corresponding hook for each of them (in this example, `self.on_setup_end()`).

Poniard includes 2 plugins out of the box.
1. Weights and Biases (`WandBPlugin`) logs configurations, metrics, plots and saves model and dataset artifacts.
2. Pandas Profiling (`PandasProfilingPlugin`) analyzes the dataset and generates an HTML report.

Extending Poniard by creating plugins is simple, as all they have to do is subclass `BasePlugin` and implement the hooks they need, leaving the rest as is.

In this instance we'll model the Adult Census dataset, a slightly more complicated classification task than in previous examples, and include both plugins.

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from poniard import PoniardClassifier
from poniard.plugins import WandBPlugin, PandasProfilingPlugin

# Adult Census dataset
X, y = fetch_openml(data_id=1590, return_X_y=True, as_frame=True)
X = X.sample(n=10000, random_state=0)
y = y.reindex(X.index)
category_cols = X.select_dtypes(include="category").columns
X = X.astype({col: object for col in category_cols})
y = pd.Series(np.where(y == ">50K", 1, 0), name="target")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

pnd = PoniardClassifier(
    numeric_threshold=80,
    n_jobs=-1,
    plugins=[WandBPlugin(project="adult-demo"), PandasProfilingPlugin()],
)
pnd.setup(X_train, y_train)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rxavier. Use `wandb login --relogin` to force relogin


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 591.25it/s]


Target info
-----------
Type: binary
Shape: (8000,)
Unique values: 2

Main metric
-----------
roc_auc

Thresholds
----------
Minimum unique values to consider a feature numeric: 80
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


/Users/rafxavier/Documents/Repos/personal/poniard/poniard/plugins/pandas_profiling.py:62: UserWarning: ipywidgets is not installed. HTML report will be saved to pandas_profiling_report.html
  warnings.warn(


,numeric,categorical_high,categorical_low,datetime
0,fnlwgt,age,education-num,
1,capital-gain,capital-loss,workclass,
2,hours-per-week,native-country,education,
3,,,marital-status,
4,,,occupation,
5,,,relationship,
6,,,race,
7,,,sex,


PoniardClassifier(estimators=None, metrics=['roc_auc', 'accuracy', 'precision', 'recall', 'f1'],
    preprocess=True, scaler=standard, numeric_imputer=simple,
    custom_preprocessor=None, numeric_threshold=80,
    cardinality_threshold=20, cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True), verbose=0,
    random_state=0, n_jobs=-1, plugins=[<poniard.plugins.wandb.WandBPlugin object at 0x16ff84550>, <poniard.plugins.pandas_profiling.PandasProfilingPlugin object at 0x2846244c0>],
    plot_options=PoniardPlotFactory())
            

As can be seen, Pandas Profiling already created a report and saved it to the default location. If `ipywidgets` is installed, the report will be included in the output.

Meanwhile, Weights and Biases either logged in or prompted for a login, and started logging information about the run (preprocessor HTML representation, dataset, inferred types). Also, because plugins can check whether other plugins are included in a Poniard estimator (by using `self._check_plugin_used()`), wandb also uploaded the profile report.

In [3]:
pnd.fit()
pnd.get_results()

DummyClassifier:  89%|████████▉ | 8/9 [00:30<00:03,  3.32s/it]               /Users/rafxavier/Envs/poniard/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rafxavier/Envs/poniard/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rafxavier/Envs/poniard/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resul

,test_roc_auc,test_accuracy,test_precision,test_recall,test_f1,fit_time,score_time
HistGradientBoostingClassifier,0.915043,0.862375,0.748392,0.645889,0.693225,0.861518,0.069380
XGBClassifier,0.914425,0.860375,0.747183,0.637063,0.687385,1.978897,0.047997
LogisticRegression,0.903243,0.848750,0.729686,0.591883,0.653078,0.195238,0.060256
SVC,0.902921,0.852000,0.738421,0.597604,0.660153,7.606432,0.317675
RandomForestClassifier,0.895694,0.845375,0.715379,0.596037,0.649987,7.460580,0.105895
KNeighborsClassifier,0.823345,0.811750,0.628024,0.536325,0.578097,0.088176,0.217586
GaussianNB,0.788458,0.548625,0.342980,0.942872,0.502522,0.106794,0.047145
DecisionTreeClassifier,0.735379,0.802250,0.586941,0.606419,0.596123,2.468232,0.045610
DummyClassifier,0.500000,0.759250,0.000000,0.000000,0.000000,0.089249,0.038644


After training has finished, WandB logs one plot per metric. However, we'll go ahead and run some additional plots.

In [4]:
pnd.plot.metrics(kind="strip", metrics=["accuracy", "roc_auc"])

Every plot produced by the PoniardPlotFactory will be logged to the wandb project and will remain an interactive Plotly plot.

In [5]:
candidates = ["XGBClassifier", "HistGradientBoostingClassifier", "LogisticRegression"]
pnd.plot.roc_curve(estimator_names=candidates)

Completed: 100%|██████████| 1/1 [00:00<00:00,  4.17it/s] 


In [6]:
pnd.plot.confusion_matrix(estimator_name="XGBClassifier")

Completed: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


In [7]:
pnd.plot.permutation_importance(estimator_name="XGBClassifier")

In [8]:
pnd.plot.partial_dependence(estimator_name="XGBClassifier", feature="marital-status")

In [9]:
pnd.plot.partial_dependence(estimator_name="XGBClassifier", feature="age")

In [10]:
xgb = pnd.get_estimator("XGBClassifier")
xgb.fit(X_train, y_train)
xgb.score(X_test, y_test)

0.862